In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Short explanations about the data.
There are 2 types of json files,
* 1 starting with "a_...." which contains the type of activity and it's timestamp (amount of microseconds since 1st January 1970)
* 1 starting with "s_...." which contains the sensors data
I made a diagram. And I also share with you a sample data of a "a_" file and a "s_" one.

In [2]:
sensor_csvpath = "../raw_data/sample/sensors.csv"
anno_csvpath = "../raw_data/sample/annotations.csv"

In [3]:
an_csv = pd.read_csv(anno_csvpath)
an_csv

,time,filename
0,2020-08-05 22:21:03,data/raw/2020-08-05-asa/a_2020-08-05-W6Ex5DZKn...


In [4]:
def an_filename(name):
    '''Get the real file name not the path from csv file'''
    filename = "a_" + name.split("a_")[1]
    return filename

In [5]:
an_csv["real_filename"] = an_csv["filename"].apply(an_filename)

In [6]:
an_csv

,time,filename,real_filename
0,2020-08-05 22:21:03,data/raw/2020-08-05-asa/a_2020-08-05-W6Ex5DZKn...,a_2020-08-05-W6Ex5DZKnuMe8eiFIsbSutgVLV72.json


In [7]:
def sen_filename(name):
    '''Get the real file name not the path from csv file'''
    filename = "s_" + name.split("s_")[1]
    return filename

In [38]:
sen_csv = pd.read_csv(sensor_csvpath)
sen_csv["real_filename"] = sen_csv["filename"].apply(sen_filename)
sen_csv.head()

,time,filename,real_filename
0,2020-08-05 22:21:01,data/raw/2020-08-05-asa/s_2020-08-05-159661889...,s_2020-08-05-1596618899641-ASUS_X013DB-W6Ex5DZ...
1,2020-08-05 22:21:01,data/raw/2020-08-05-asa/s_2020-08-05-159661890...,s_2020-08-05-1596618902316-SHV40-W6Ex5DZKnuMe8...
2,2020-08-05 22:21:01,data/raw/2020-08-05-asa/s_2020-08-05-159661890...,s_2020-08-05-1596618909689-ASUS_X013DB-W6Ex5DZ...
3,2020-08-05 22:21:01,data/raw/2020-08-05-asa/s_2020-08-05-159661891...,s_2020-08-05-1596618912434-SHV40-W6Ex5DZKnuMe8...
4,2020-08-05 22:21:01,data/raw/2020-08-05-asa/s_2020-08-05-159661891...,s_2020-08-05-1596618919758-ASUS_X013DB-W6Ex5DZ...


In [9]:
def key(X):
    key, value = list(X.items())[0]
    return key.strip("()")

In [10]:
def value(X):
    key, value = list(X.items())[0]
    key2, value2 = list(value[0].items())[0]
    return value2

In [11]:
def process(X):
    key, value = list(X.items())[0]
    key2, value2 = list(value[0].items())[0]
    return key2

In [13]:
anno_jsonpath = "../raw_data/sample/annot/a_2020-08-05-W6Ex5DZKnuMe8eiFIsbSutgVLV72.json"
an_json = pd.read_json(anno_jsonpath)
an_json.head()

,dt,os,p,pf,uid,v
0,{'down_stairs': [{'Begin': 1596618985598}]},26,SHV40,1,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1
1,{'down_stairs': [{'End': 1596618989833}]},26,SHV40,1,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1
2,{'down_stairs': [{'Begin': 1596618995255}]},26,SHV40,1,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1
3,{'down_stairs': [{'End': 1596618997833}]},26,SHV40,1,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1
4,{'down_stairs': [{'Begin': 1596619066247}]},26,SHV40,1,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1


In [39]:
an_json["activities"] = an_json["dt"].apply(key)
an_json["process"] = an_json["dt"].apply(process)
an_json["time"] = an_json["dt"].apply(value)
an_json["real_time"] = an_json["time"].apply(convert_to_realtime)

In [40]:
an_json.head()

,dt,os,p,pf,uid,v,activities,process,time,real_time
0,{'down_stairs': [{'Begin': 1596618985598}]},26,SHV40,1,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1,down_stairs,Begin,1596618985598,19:03:09:85
1,{'down_stairs': [{'End': 1596618989833}]},26,SHV40,1,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1,down_stairs,End,1596618989833,19:03:09:89
2,{'down_stairs': [{'Begin': 1596618995255}]},26,SHV40,1,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1,down_stairs,Begin,1596618995255,19:03:09:95
3,{'down_stairs': [{'End': 1596618997833}]},26,SHV40,1,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1,down_stairs,End,1596618997833,19:03:09:97
4,{'down_stairs': [{'Begin': 1596619066247}]},26,SHV40,1,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1,down_stairs,Begin,1596619066247,19:03:10:66


In [41]:
final_an_csv = an_json[an_json.columns[-4:]]

In [ ]:
# final_an_csv.to_csv("clean_an.csv",index=False)

In [27]:
sen_jsonpath = "../raw_data/sample/sens/s_2020-08-05-1596618922655-SHV40-W6Ex5DZKnuMe8eiFIsbSutgVLV72.json"
sen_json = pd.read_json(sen_jsonpath)
sen_json.head()

,dt,os,pm,p,uid,v
a,"[[1596618912439.9998, 864412474877.0, 1.177947...",26,SHV40,0,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1
g,"[[1596618912441.0, 864413379825.0, 0.021991148...",26,SHV40,0,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1
p,[],26,SHV40,0,W6Ex5DZKnuMe8eiFIsbSutgVLV72,1


In [29]:
sen_acc = np.array(sen_json.dt["a"])
acc_df = pd.DataFrame(sen_acc, columns=["time_stamp", "sensor_time_stamp","X","Y","Z"])
acc_df.head()

,time_stamp,sensor_time_stamp,X,Y,Z
0,1.596619e+12,8.644125e+11,1.177947,2.901772,9.040505
1,1.596619e+12,8.644323e+11,1.359907,2.873042,9.212888
2,1.596619e+12,8.644523e+11,1.359907,2.920926,9.117120
3,1.596619e+12,8.644725e+11,1.455675,2.987964,9.097966
4,1.596619e+12,8.644925e+11,1.446098,3.045424,9.184157


In [31]:
sen_gyro = np.array(sen_json.dt["g"])
gyro_df = pd.DataFrame(sen_gyro, columns=["time_stamp", "sensor_time_stamp","X","Y","Z"])
gyro_df.head()

,time_stamp,sensor_time_stamp,X,Y,Z
0,1.596619e+12,8.644134e+11,0.021991,-0.100182,0.162490
1,1.596619e+12,8.644334e+11,0.014661,-0.096517,0.125838
2,1.596619e+12,8.644534e+11,0.056200,-0.053756,0.107512
3,1.596619e+12,8.644734e+11,0.040317,-0.078191,0.073304
4,1.596619e+12,8.644934e+11,0.002443,-0.124617,0.013439


In [ ]:
def convert(seconds): 
    milli = (seconds % 1) * 100
    seconds = seconds % (24 * 3600) 
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
      
    return "%d:%02d:%02d:%02d" % (hour, minutes, seconds, milli)

In [16]:
def convert_to_realtime(timestamp): 
    seconds = timestamp/100000
    milli = (seconds % 1) * 100
    seconds = seconds % (24 * 3600) 
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60 
    seconds %= 60 
    return "%d:%02d:%02d:%02d" % (hour, minutes, seconds, milli) 
    #return "%d:%02d:%02d" % (hour, minutes, seconds)

In [32]:
gyro_df["real_time_stamp"] = gyro_df["time_stamp"].apply(convert_to_realtime)

In [44]:
gyro_df.head()

,time_stamp,sensor_time_stamp,X,Y,Z,real_time_stamp
0,1.596619e+12,8.644134e+11,0.021991,-0.100182,0.162490,19:03:09:12
1,1.596619e+12,8.644334e+11,0.014661,-0.096517,0.125838,19:03:09:12
2,1.596619e+12,8.644534e+11,0.056200,-0.053756,0.107512,19:03:09:12
3,1.596619e+12,8.644734e+11,0.040317,-0.078191,0.073304,19:03:09:12
4,1.596619e+12,8.644934e+11,0.002443,-0.124617,0.013439,19:03:09:12


In [46]:
final_an_csv.tail(20)

,activities,process,time,real_time
294,climb_stairs,Begin,1596632651833,19:05:26:51
295,climb_stairs,End,1596632654922,19:05:26:54
296,climb_stairs,Begin,1596632660050,19:05:26:60
297,climb_stairs,End,1596632663131,19:05:26:63
298,down_stairs,Begin,1596632670653,19:05:26:70
299,down_stairs,End,1596632674407,19:05:26:74
300,down_stairs,Begin,1596632679891,19:05:26:79
301,down_stairs,End,1596632682812,19:05:26:82
302,down_stairs,Begin,1596632687620,19:05:26:87
303,down_stairs,End,1596632690787,19:05:26:90


### Q and A


- How do we pick the data? Since we have multiple files for each second <<< No 
- The special requirement : Wed 12/2
- How to upload model OwO : Wed 12/2

Take 

In [37]:
final_df = pd.DataFrame([], columns=["Activity","Start","End","Acc_Time_stamp","Acc_X","Acc_Y","Acc_Z"])
final_df

,Activity,Start,End,Acc_Time_stamp,Acc_X,Acc_Y,Acc_Z


- Funtion for merging
- Create a Activity , Start , End (2s) >>